# 根据要求导出`WRF-CMAQ`模拟数据
2、	四个国控点的数据。各指标如O3浓度、PM2.5浓度、NO2浓度、各过程贡献、各来源贡献和敏感性指标（HNO3/H2O2比值）的时间序列，可以存成一个excel表格，例如第一列是时间（小时），第二列是O3浓度，第三列….。

---
*@author: Evan*\
*@date: 2023-05-14*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.io import savemat

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../src/')
from findpoint import findpoint
import ModelEvaLib as me

In [2]:
chemFile = xr.open_dataset('F:/Data/case_anqing/April/April_chem.nc')
metFile  = xr.open_dataset('F:/Data/case_anqing/April/April_met.nc')
isam = xr.open_dataset('F:/Data/case_anqing/April/April_ISAM1.nc')
pa=xr.open_dataset('F:/Data/case_anqing/April/April_PA1.nc')

lat = chemFile.latitude
lon = chemFile.longitude

ncfile = chemFile.O3

nlevel = 21
O3   = chemFile.O3[:,:nlevel,:,:]
NO2  = chemFile.NO2[:,:nlevel,:,:]
PM25 = chemFile.PM25[:,:nlevel,:,:]
ht   = chemFile.HT[:,:nlevel,:,:]

HADV = pa.HADV_O3[:,:nlevel,:,:]
ZADV = pa.ZADV_O3[:,:nlevel,:,:]
HDIF = pa.HDIF_O3[:,:nlevel,:,:]
VDIF = pa.VDIF_O3[:,:nlevel,:,:]
DDEP = pa.DDEP_O3[:,:nlevel,:,:]
CHEM = pa.CHEM_O3[:,:nlevel,:,:]

AQ  = isam.O3_AQ[:,:nlevel,:,:]
HF  = isam.O3_HF[:,:nlevel,:,:]
TL  = isam.O3_TL[:,:nlevel,:,:]
CZ  = isam.O3_CZ[:,:nlevel,:,:]
JJ  = isam.O3_JJ[:,:nlevel,:,:]
OTH = isam.O3_OTH[:,:nlevel,:,:]
BCO = isam.O3_BCO[:,:nlevel,:,:]
ICO = isam.O3_ICO[:,:nlevel,:,:]

HNO3 = pa.HNO3prod[:,:nlevel,:,:]
H2O2 = pa.H2O2prod[:,:nlevel,:,:]

In [3]:
siteloc = pd.read_excel('F:/Data/case_anqing/April/obsdata/安庆市站点坐标.xlsx',header=0,index_col=0)

sites = ['安庆大学','市人大','环科院','马山宾馆','安庆政务中心']

lono = siteloc[siteloc.index.isin(sites)]['经度']
lato = siteloc[siteloc.index.isin(sites)]['纬度']

lono

站点名称
安庆大学      116.9898
市人大       117.0597
环科院       117.0807
马山宾馆      117.0331
安庆政务中心    117.1150
Name: 经度, dtype: float64

In [4]:
x_index=[]
y_index=[]

for name in sites:
    out_x, out_y = findpoint(lono[name],lato[name],ncfile)
    x_index.append(out_x)
    y_index.append(out_y)
O3_sel = O3.isel(x=x_index, y=y_index)
NO2_sel  = NO2.isel(x=x_index, y=y_index)
PM25_sel = PM25.isel(x=x_index, y=y_index)
htsel    = ht.isel(x=x_index, y=y_index)

HADV_sel  = HADV.isel(x=x_index, y=y_index)
ZADV_sel  = ZADV.isel(x=x_index, y=y_index)
HDIF_sel  = HDIF.isel(x=x_index, y=y_index)
VDIF_sel  = VDIF.isel(x=x_index, y=y_index)
DDEP_sel  = DDEP.isel(x=x_index, y=y_index)
CHEM_sel  = CHEM.isel(x=x_index, y=y_index)

HNO3_sel = HNO3.isel(x=x_index, y=y_index)
H2O2_sel = H2O2.isel(x=x_index, y=y_index)

AQ_sel  = AQ.isel(x=x_index, y=y_index)
HF_sel  = HF.isel(x=x_index, y=y_index)
TL_sel  = TL.isel(x=x_index, y=y_index)
CZ_sel  = CZ.isel(x=x_index, y=y_index)
JJ_sel  = JJ.isel(x=x_index, y=y_index)
OTH_sel = OTH.isel(x=x_index, y=y_index)
BCO_sel = BCO.isel(x=x_index, y=y_index)
ICO_sel = ICO.isel(x=x_index, y=y_index)

In [6]:
diag_O3   = np.diagonal(O3_sel, axis1=2, axis2=3)
diag_NO2  = np.diagonal(NO2_sel, axis1=2, axis2=3)
diag_PM25 = np.diagonal(PM25_sel, axis1=2, axis2=3)
diag_ht   = np.diagonal(htsel, axis1=2, axis2=3)

In [7]:
diag_HADV  = np.diagonal(HADV_sel, axis1=2, axis2=3)
diag_ZADV  = np.diagonal(ZADV_sel, axis1=2, axis2=3)
diag_HDIF  = np.diagonal(HDIF_sel, axis1=2, axis2=3)
diag_VDIF  = np.diagonal(VDIF_sel, axis1=2, axis2=3)
diag_DDEP  = np.diagonal(DDEP_sel, axis1=2, axis2=3)
diag_CHEM  = np.diagonal(CHEM_sel, axis1=2, axis2=3)

In [9]:
diag_AQ  = np.diagonal(AQ_sel, axis1=2, axis2=3)
diag_HF  = np.diagonal(HF_sel, axis1=2, axis2=3)
diag_TL  = np.diagonal(TL_sel, axis1=2, axis2=3)
diag_CZ  = np.diagonal(CZ_sel, axis1=2, axis2=3)
diag_JJ  = np.diagonal(JJ_sel, axis1=2, axis2=3)
diag_OTH = np.diagonal(OTH_sel, axis1=2, axis2=3)
diag_BCO = np.diagonal(BCO_sel, axis1=2, axis2=3)
diag_ICO = np.diagonal(ICO_sel, axis1=2, axis2=3)

In [10]:
savemat(
    'D:/Download/points.mat',
    {
    'height':diag_ht,
    'O3':diag_O3,
    'NO2':diag_NO2,
    'PM25':diag_PM25,
    'HADV':diag_HADV,
    'ZADV':diag_ZADV,
    'HDIF':diag_HDIF,
    'VDIF':diag_VDIF,
    'DDEP':diag_DDEP,
    'CHEM':diag_CHEM,
    'AQ':diag_AQ,
    'HF':diag_HF,
    'TL':diag_TL,
    'CZ':diag_CZ,
    'JJ':diag_JJ,
    'OTH':diag_OTH,
    'BCO':diag_BCO,
    'ICO':diag_ICO
    }
    )